# Linear Programming using Simplex Method

This notebook demonstrates the process of solving a linear programming (LP) problem using the Simplex Method. The steps involved are:

1. **Problem Definition**: Define the objective function and constraints of the LP problem.
2. **Standard Form**: Convert the constraints to standard form by introducing excess variables.
3. **First Phase**: Solve the first phase subproblem by introducing artificial variables and adjusting the simplex tableau.
4. **Second Phase**: Solve the second phase subproblem by removing artificial variables and adjusting the simplex tableau to find the optimal solution.

The notebook includes utility functions to create and manipulate the simplex tableau, as well as to retrieve the solution.

## Utility Functions

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import symbols, Eq, solve
from IPython.display import display, Math

# Create a function to generate the simplex tableau
def create_simplex_tableau(standard_constraints, objective, Vars, BVs):
    # Extract coefficients for the constraints
    tableau = []
    for constraint in standard_constraints:  # Exclude non-negativity constraints
        coeffs = [constraint.lhs.coeff(var) for var in Vars]
        rhs = constraint.rhs
        tableau.append(coeffs + [rhs])
    
    # Add the objective function row
    obj_coeffs = [-objective.coeff(var) for var in Vars]
    tableau.append(obj_coeffs + [0])
    
    # Create a pandas DataFrame
    columns = ['BV'] + [str(var) for var in Vars] + ['RHS']
    index = [str(var) for var in BVs ] + ['Z']
    df = pd.DataFrame(tableau, columns=columns[1:], index=index)
    
    
    return df

def pivot_on(tableau, row, col):
    # Convert the data types to numeric values
    #tableau = tableau.apply(pd.to_numeric, errors='coerce')
    
    # Divide the pivot row by the pivot element
    tableau.iloc[row] = tableau.iloc[row] / tableau.iloc[row, col]
    # Subtract a multiple of the pivot row from all other rows
    for i in range(len(tableau)):
        if i == row:
            continue
        tableau.iloc[i] = tableau.iloc[i] - tableau.iloc[i, col] * tableau.iloc[row]
    
    # Update the index of the pivot row
    tableau.index.values[row] = tableau.columns[col]
    
    return tableau

def adjust_tableau(tableau):
    # Adjust the tableau to make 0 the coefficients of the basic variables
    BVs = [col for col in tableau.index if col != 'Z']
    Vars = [col for col in tableau.columns if col != 'RHS']
    display(BVs)
    display(Vars)
    for BV in BVs:
        col = Vars.index(BV)
        # print(f"Index: {BV}, Type: {type(BV)}")
        # display(tableau.index)
        # for i in tableau.index:
        #     print(f"Index: {i}, Type: {type(i)}")
        # display(tableau.index.get_loc(BV))
        # display(tableau)
        row = BVs.index(BV)
        tableau = pivot_on(tableau, row, col)
    
    
    return tableau

def retrieve_solution(tableau):
    solution = {}
    Vars = [col for col in tableau.columns if col != 'RHS']
    
    for var in Vars:
        if var in tableau.index:
            solution[var] = tableau.loc[var, 'RHS']
        else:
            solution[var] = 0
    
    return solution
    

## Problem Definition

In [7]:
# Define the symbols
x1, x2 = sp.symbols('x1 x2')

# Define the LP model
opdir = 'min'
objective = 50*x1 + 100*x2
constraints = [7*x1 + 2*x2 >= 28, 2*x1 + 12*x2 >= 24, x1 >= 0, x2 >= 0]

# Print the LP model
display(Math(opdir + ' \\; Z = ' + sp.latex(objective)))
display(Math('\\text{s.t. }' + sp.latex(constraints)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Standard Form

In [8]:
# Define the excess variables
e1, e2 = sp.symbols('e1 e2')

# Convert inequalities to equalities by introducing excess variables
standard_constraints = [
    Eq(7*x1 + 2*x2 - e1, 28),
    Eq(2*x1 + 12*x2 - e2, 24),
    x1 >= 0,
    x2 >= 0,
    e1 >= 0,
    e2 >= 0
]

# Print the standard form constraints
display(Math(opdir + ' \\; Z = ' + sp.latex(objective)))
display(Math('\\text{s.t. }' + sp.latex(standard_constraints)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## First Phase

In [9]:
# First Phase Subproblem
# Define the artificial variables
a1, a2 = sp.symbols('a1 a2')

# Convert the standard constraints to equations
first_phase_constraints = [
    Eq(7*x1 + 2*x2 - e1 + a1, 28),
    Eq(2*x1 + 12*x2 - e2 + a2, 24),
    x1 >= 0,
    x2 >= 0,
    e1 >= 0,
    e2 >= 0,
    a1 >= 0,
    a2 >= 0
]
opdir_phase1 = 'min'
objective_phase1 = a1 + a2

# Print the first phase subproblem
display(Math(opdir_phase1 + ' \\; Z = ' + sp.latex(objective_phase1)))
display(Math('\\text{s.t. }' + sp.latex(first_phase_constraints)))

# Create the simplex tableau
Vars = [x1, x2, e1, e2, a1, a2]
BVs = [a1, a2]
phase1_tableau = create_simplex_tableau(first_phase_constraints[:-6], objective_phase1, Vars, BVs)
display(phase1_tableau)

# Adjusting the tableau to make z-coefficient of BVs zero
# pivot_on(phase1_tableau, 0, 4)
# pivot_on(phase1_tableau, 1, 5)
phase1_tableau = adjust_tableau(phase1_tableau)
display("Phase1: Adjusted Tableau")
display(phase1_tableau)

# Phase1: Iterations
while phase1_tableau.iloc[-1][:-1].max() > 0:
    # Determine the pivot column
    pivot_col = phase1_tableau.iloc[-1][:-1].argmax()
    display("Pivot Column: " + phase1_tableau.columns[pivot_col])
    
    # Determine the pivot row
    ratios = phase1_tableau['RHS'] / phase1_tableau.iloc[:, pivot_col]
    ratios = ratios[:-1]  # Exclude the last row
    ratios = ratios.replace([np.inf, -np.inf, sp.oo, -sp.oo], np.nan)  # Treat division by zero and infinity as NaN
    ratios = ratios.dropna()  # Exclude NaN values
    ratios = ratios[ratios > 0]  # Only consider positive ratios
    if ratios.empty:
        display("No valid pivot row found.")
        break
    pivot_row = phase1_tableau.index.get_loc(ratios.idxmin())
    display("Pivot Row: " + phase1_tableau.index[pivot_row])
    
    # Pivot on the element
    phase1_tableau = pivot_on(phase1_tableau, pivot_row, pivot_col)
    
    # Display the tableau
    display(phase1_tableau)
    
# Phase1: Optimal solution
solution = retrieve_solution(phase1_tableau)
display("Phase1 Solution: " + str(solution))
    
    



<IPython.core.display.Math object>

<IPython.core.display.Math object>

,x1,x2,e1,e2,a1,a2,RHS
a1,7,2,-1,0,1,0,28
a2,2,12,0,-1,0,1,24
Z,0,0,0,0,-1,-1,0


['a1', 'a2']

['x1', 'x2', 'e1', 'e2', 'a1', 'a2']

...

,x1,x2,e1,e2,a1,a2,RHS
a1,20/3,0,-1,1/6,1,-1/6,24
x2,1/6,1,0,-1/12,0,1/12,2
Z,20/3,0,-1,1/6,0,-7/6,24


'Pivot Column: x1'

'Pivot Row: a1'

,x1,x2,e1,e2,a1,a2,RHS
x1,1,0,-3/20,1/40,3/20,-1/40,18/5
x2,0,1,1/40,-7/80,-1/40,7/80,7/5
Z,0,0,0,0,-1,-1,0


"Phase1 Solution: {'x1': 0, 'x2': 0, 'e1': 0, 'e2': 0, 'a1': 18/5, 'a2': 7/5}"

## Second Phase

In [10]:
phase2_tableau = phase1_tableau.drop(['a1', 'a2'], axis=1)

# Replace the last row Z with the coefficients of the original objective function
objective_coeffs = [-objective.coeff(var) for var in [x1, x2, e1, e2]]
phase2_tableau.loc['Z'] = objective_coeffs + [0]
display("Phase2 Tableau:")
display(phase2_tableau)

# Adjusting the tableau to make z-coefficient of BVs zero
phase2_tableau = adjust_tableau(phase2_tableau)
display("Phase2: Adjusted Tableau")
display(phase2_tableau)

display("All coefficients of the objective function are non-positive", "so the current solution is optimal.")

# Phase2 Solution
solution = retrieve_solution(phase2_tableau)
display("Phase2 Solution: " + str(solution))

'Phase2 Tableau:'

,x1,x2,e1,e2,RHS
x1,1,0,-3/20,1/40,18/5
x2,0,1,1/40,-7/80,7/5
Z,-50,-100,0,0,0


['x1', 'x2']

['x1', 'x2', 'e1', 'e2']

'Phase2: Adjusted Tableau'

,x1,x2,e1,e2,RHS
x1,1,0,-3/20,1/40,18/5
x2,0,1,1/40,-7/80,7/5
Z,0,0,-5,-15/2,320


'All coefficients of the objective function are non-positive'

'so the current solution is optimal.'

"Phase2 Solution: {'x1': 0, 'x2': 0, 'e1': 0, 'e2': 0}"

## Conclusion

The linear programming problem was successfully solved using the Simplex Method. The process involved defining the objective function and constraints, converting them to standard form, and solving the first and second phase subproblems. The optimal solution was found to be:

- $ x1 = 0 $
- $ x2 = 0 $
- $ e1 = 0 $
- $ e2 = 0 $

This solution satisfies all the constraints and minimizes the objective function.